# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os

from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 16

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123
                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!= 0:
            batch_data = np.zeros((batch_size,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder for rem images
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in the folder for rem images
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123
                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here
model = Sequential()

model.add(Conv3D(64, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(15,84,84,3),
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(128, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Conv3D(128, #Number of filters 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())

model.add(Activation('elu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(256, #Number pf filters 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())

model.add(Activation('elu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(125, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))


#softmax layer
model.add(Dense(5, activation='softmax'))




Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.SGD(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 15, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 84, 84, 64)    256       
_________________________________________________________________
activation (Activation)      (None, 15, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 42, 42, 64)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 7, 42, 42, 128)    221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 42, 42, 128)    512       
_________________________________________________________________
activation_1 (Activation)    (None, 7, 42, 42, 128)    0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, cooldown=0, verbose=1,min_lr = 0)
callbacks_list = [checkpoint, LR]

W0510 07:53:57.783280 140231089870656 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0510 07:54:35.226814 140231089870656 deprecation.py:323] From <ipython-input-12-bd77c9c60c14>:3: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]


Source path =  Project_data/train ; batch size = 16


/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch 1/25
40/42 [===========================>..] - ETA: 5s - loss: 1.7892 - categorical_accuracy: 0.2203

/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


42/42 [==============================] - ETA: 0s - loss: 1.7845 - categorical_accuracy: 0.2232Source path =  Project_data/val ; batch size = 16

Epoch 00001: val_loss improved from inf to 1.54416, saving model to model_init_2021-05-1007_53_23.548955/model-00001-1.78453-0.22321-1.54416-0.22321.h5
42/42 [==============================] - 142s 3s/step - loss: 1.7845 - categorical_accuracy: 0.2232 - val_loss: 1.5442 - val_categorical_accuracy: 0.2232 - lr: 0.0010
Epoch 2/25
42/42 [==============================] - ETA: 0s - loss: 1.5178 - categorical_accuracy: 0.3199
Epoch 00002: val_loss improved from 1.54416 to 1.51225, saving model to model_init_2021-05-1007_53_23.548955/model-00002-1.51777-0.31994-1.51225-0.24107.h5
42/42 [==============================] - 57s 1s/step - loss: 1.5178 - categorical_accuracy: 0.3199 - val_loss: 1.5123 - val_categorical_accuracy: 0.2411 - lr: 0.0010
Epoch 3/25
42/42 [==============================] - ETA: 0s - loss: 1.4276 - categorical_accuracy: 0.3750
Ep

Epoch 22/25
42/42 [==============================] - ETA: 0s - loss: 0.5768 - categorical_accuracy: 0.8036
Epoch 00022: val_loss improved from 0.73554 to 0.68797, saving model to model_init_2021-05-1007_53_23.548955/model-00022-0.57675-0.80357-0.68797-0.70536.h5
42/42 [==============================] - 54s 1s/step - loss: 0.5768 - categorical_accuracy: 0.8036 - val_loss: 0.6880 - val_categorical_accuracy: 0.7054 - lr: 0.0010
Epoch 23/25
42/42 [==============================] - ETA: 0s - loss: 0.6140 - categorical_accuracy: 0.7798
Epoch 00023: val_loss did not improve from 0.68797
42/42 [==============================] - 54s 1s/step - loss: 0.6140 - categorical_accuracy: 0.7798 - val_loss: 0.7243 - val_categorical_accuracy: 0.6786 - lr: 0.0010
Epoch 24/25
42/42 [==============================] - ETA: 0s - loss: 0.4718 - categorical_accuracy: 0.8423
Epoch 00024: val_loss did not improve from 0.68797
42/42 [==============================] - 54s 1s/step - loss: 0.4718 - categorical_accurac